In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from quantum_algorithms import QuantumPhaseEstimation
from qiskit_aer import Aer

### $\text{Unitary Operator for Validation Test}$

In [2]:
# Control-X in matrix form
unitary_matrix = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
], dtype=complex)

### $\text{Classical Approach}$

In [3]:
# Determine the eigenvalues and eigenvectors using NumPy
eigenvalues_exact, eigenvectors_exact = np.linalg.eig(unitary_matrix)

# Logging
for val, vec in zip(eigenvalues_exact, eigenvectors_exact):
    print(f"""Eigenvector: {vec} ===> Eigenvalue: {val:.4f}""") 

Eigenvector: [-0.+0.j -0.+0.j  1.+0.j  0.+0.j] ===> Eigenvalue: 1.0000+0.0000j
Eigenvector: [-0.+0.j -0.+0.j  0.+0.j  1.+0.j] ===> Eigenvalue: -1.0000+0.0000j
Eigenvector: [0.70710678-0.j 0.70710678+0.j 0.        +0.j 0.        +0.j] ===> Eigenvalue: 1.0000+0.0000j
Eigenvector: [ 0.70710678+0.j -0.70710678+0.j  0.        +0.j  0.        +0.j] ===> Eigenvalue: 1.0000+0.0000j


### $\text{Quantum Approach}$

In [4]:
# Qubit initialization
n_control = 3
n_target = np.log2(unitary_matrix.shape[0])

# Build a quantum circuit
qc = QuantumCircuit(n_control + n_target, n_control)

# Initialize the eigenstate |11> = [0. 0. 0. 1.]
qc.x([3, 4])

# Apply the Quantum Phase Estimation
QuantumPhaseEstimation(qc, unitary_matrix, n_control)

# Measurement
for order in range(n_control):
    qc.measure(order, order)

# Draw the circuit
qc.draw()

░ ┌───┐                                           ░     ░ ┌───┐»
q_0: ──────░─┤ H ├──────■────────────────────────────────────░──X──░─┤ H ├»
           ░ ├───┤      │                                    ░  │  ░ └───┘»
q_1: ──────░─┤ H ├──────┼─────────────■──────────────────────░──┼──░──────»
           ░ ├───┤      │             │                      ░  │  ░      »
q_2: ──────░─┤ H ├──────┼─────────────┼─────────────■────────░──X──░──────»
     ┌───┐ ░ └───┘┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐ ░     ░      »
q_3: ┤ X ├─░──────┤0           ├┤0           ├┤0           ├─░─────░──────»
     ├───┤ ░      │  unitary^1 ││  unitary^2 ││  unitary^4 │ ░     ░      »
q_4: ┤ X ├─░──────┤1           ├┤1           ├┤1           ├─░─────░──────»
     └───┘ ░      └────────────┘└────────────┘└────────────┘ ░     ░      »
c: 3/═════════════════════════════════════════════════════════════════════»
                                                                          »
«                                           ░ ┌─┐      
«q_0: ─■────────■───────────────────────────░─┤M├──────
«      │        │P(π/4) ┌───┐               ░ └╥┘┌─┐   
«q_1: ─┼────────■───────┤ H ├─■─────────────░──╫─┤M├───
«      │P(π/2)          └───┘ │P(π/2) ┌───┐ ░  ║ └╥┘┌─┐
«q_2: ─■──────────────────────■───────┤ H ├─░──╫──╫─┤M├
«                                     └───┘ ░  ║  ║ └╥┘
«q_3: ──────────────────────────────────────░──╫──╫──╫─
«                                           ░  ║  ║  ║ 
«q_4: ──────────────────────────────────────░──╫──╫──╫─
«                                           ░  ║  ║  ║ 
«c: 3/═════════════════════════════════════════╩══╩══╩═
«                                              0  1  2

In [5]:
# Simulation
simulator = Aer.get_backend("qasm_simulator")
compiled = transpile(qc, simulator)
job = simulator.run(compiled, shots=1000)
result = job.result()

# Show the result of simulation
counts = result.get_counts()
print(counts)

{'000': 514, '100': 486}


In [6]:
# Logging
theta = int(str(max(counts)), 2) / 2**(n_control)
power = 2 * np.pi * theta
eigenvalues_predict = complex(np.cos(power), np.sin(power))

print(f"Eigenstate: [0. 0. 0. 1.] ===> Eigenvalue: {eigenvalues_predict:.4f}")

Eigenstate: [0. 0. 0. 1.] ===> Eigenvalue: -1.0000+0.0000j
